In [ ]:
##### FOR MULTICLASS CLASSIFICATION ( MORE THAN 10)

Your data must me in this way:--
                               
                          |-----1(folder)
             ----train----                  ( add more classes of folder)
                          |-----2(folder)
             
    Data----
    
                           |-----1(folder)
             ----test-----                    (add more classes of folder)
                           |------2(folder)

In [ ]:
#importing import library
import numpy as np
import tensorflow as tf
from tensorflow import keras
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.layers import Input, Lambda, Dense, Flatten,Dropout
from keras.models import Model
from keras.optimizers import Adam

In [ ]:
#for data argumentation
train_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [ ]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [ ]:
batch_size=30  #batch size

In [ ]:
# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'data/Traning_data',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='categorical',  # since we use categorical_crossentropy loss, we need binary labels
        color_mode="rgb")    #for colored image for black and white  use grayscale



In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
augmented_images_train = [train_generator[0][0][0] for i in range(5)] #change the value and see effect

In [ ]:
# Re-use the same custom plotting function defined and used
# above to visualize the training images
plotImages(augmented_images_train)

In [ ]:
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        'data/Test_data',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='categorical',
        color_mode="rgb")

In [ ]:
augmented_images_val = [validation_generator[0][0][0] for i in range(5)]

In [ ]:
# above to visualize the validation images
plotImages(augmented_images_val)

In [ ]:
IMAGE_SIZE=[150,150]

In [ ]:
# add preprocessing layer to the front of VGG
vgg = VGG16(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [ ]:
vgg.summary()

In [ ]:
#Transfer learning
for layer in vgg.layers:
    layer.trainable = False
    
    
x = Flatten()(vgg.output)
x = Dense(1000, activation='relu')(x)
x = Dropout(0.2)(x)
prediction = Dense(36, activation='softmax')(x)  #here i have used 36 classes you may change

# create a model object
model = Model(inputs=vgg.input, outputs=prediction)    

In [ ]:
# view the structure of the model
model.summary()


In [ ]:
# tell the model what cost and optimization method to use
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
steps_per_epoch=2000
validation_steps=800
epochs=90

#training the model
model.fit_generator(
        train_generator,
        steps_per_epoch=steps_per_epoch // batch_size,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_steps // batch_size)

In [ ]:
#to save model for future use
model.save('model.h5')

In [ ]:
# loss
plt.plot(model.history.history['loss'], label='train loss')
plt.plot(model.history.history['val_loss'], label='val loss')
plt.legend()
plt.show()
plt.savefig('LossVal_loss')

# accuracies
plt.plot(model.history.history['accuracy'], label='train acc')
plt.plot(model.history.history['val_accuracy'], label='val acc')
plt.legend()
plt.show()
plt.savefig('AccVal_acc')


#####FOCUS

In [ ]:
path="data/Test_data/1/2867.png"  #enter  path of predecting image

from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
test=load_img(path,grayscale=False,target_size=(150,150))
test1 = img_to_array(test)
test1.shape
import matplotlib.pyplot as plt

plt.imshow(test1.reshape(150,150,3))
test1=test1/255


y=model.predict(test1.reshape(1,150,150,3))
p=np.argmax(y)
print(p)
a[p]

In [ ]:
from tensorflow import keras
import numpy as np

model = keras.models.load_model('katogha.h5')  #to load the model